# Serwery Model Context Protocol

Model Context Protocol (MCP) to mechanizm zapewniający LLMom dostęp do narzędzi i funkcji, oraz informacji z zewnętrznych systemów, które mogą one wykorzystywać przez mechanizm Function Calling. Przy czym, jako że jest to protokół ustandaryzowany, umożliwia on korzystanie z jednego serwera przez modele i asystentów od różnych dostawców.

W tym notebooku mamy trzy różne podejścia do wykorzystania MCP:
1. **Bezpośrednia komunikacja z OpenAI API i serwerem MCP** - implementacja niskopoziomowa
2. **Pydantic AI** - połączenie agenta Pydantic AI z serwerem MCP
3. **Open AI agents** - połączenie agenta Open AI Agents z serwerem MCP

Każde podejście demonstruje, jak modele mogą wykorzystywać narzędzia / funkcje udostępniane przez serwer MCP do wykonywania zadań wykraczających poza ich standardowe możliwości.

---

Na końcu notebooka znajduje się również przykład niskopoziomowego korzystania z zasobów udostępnianych przez serwer MCP.

## Podejście niskopoziomowe

Komunikację z serwerem MCP obsługujemy niskopoziomowo:
- sami pobieramy listę narzędzi z serwera MCP
- sami tłumaczymy opis dostępny funkcji z formatu MCP, na format oczekiwany przez modele Open AI
- sami wykonujemy funkcje na serwerze MCP i przekazujemy wynik z powrotem do modelu

In [ ]:
from mcp import ClientSession
from mcp.client.sse import sse_client
import json
from openai import OpenAI
import os

In [ ]:
# Adres serwera MCP
mcp_url = "http://mcp:8801/sse"

# Aby korzysztać z lokalnego serwera MCP, uruchom go w terminalu wspisując:
# > export MCP_PORT=1234
# > python mcp_scripts/math_functions.py 
# i odkomentuj poniższą linię
# mcp_url = "http://localhost:1234/sse"

In [ ]:
async def mcp_list_tools_lowlevel(mcp_url):
    """
    Pobiera listę dostępnych narzędzi z serwera MCP.

    Parametry:
    mcp_url (str): URL serwera MCP.

    Zwraca:
    list: Lista dostępnych narzędzi w formacie zwracanym przez serwer MCP.
    """
    # Nawiązanie połączenia SSE
    async with sse_client(url=mcp_url) as (read, write):
        # Utworzenie sesji klienta
        async with ClientSession(read, write) as session:
            # Inicjalizacja połączenia
            await session.initialize()

            # Pobranie listy dostępnych narzędzi
            tools_response = await session.list_tools()

            return tools_response.tools

In [ ]:
# Pobranie listy dostępnych narzędzu (wywołanie funkcji asynchronicznej)
tools = await mcp_list_tools_lowlevel(mcp_url)

# Jeśli chcesz uruchomić to w kontekście nie-asynchronicznym, możesz użyć asyncio.run()
# Gdy uruchamiamy w Jupyter Notebooku, to nie jest konieczne, bo on już działa w pętli asynchronicznej
# 
# import asyncio
# tools = asyncio.run(mcp_list_tools_lowlevel())

In [ ]:
mcp_tools_json = json.dumps([tool.model_dump() for tool in tools], indent=4)

print(mcp_tools_json)

Format opisu narzędzi zwracany przez serwer MCP jest inny niż format oczekiwany przez modele od Open AI. Musimy więc ten opis przekonwertować. W tym przykładzie robimy to użzywając prompta. Jednak, w zastosowaniach produkcyjnych poprawniej jest zrobić to z wykorzystaniem dedykowanej logiki tłumaczącej jeden z formatów na drugi.

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

conversion_prompt = f"""
    Przekonwertuj poniższy dokument JSON opisujący toole zgodnie z formatem Model Context Protocol na format zgodny z OpenAI Tools Schema.
    
    Opis narzędzii ma następujący format:
    {{
        "tools" = [
            {{
                "type": "function",
                "function": {{... opis danej funckji ...}}
            }},
            {{
                "type": "function",
                "function": {{... opis danej funckji ...}}
            }}
        ]
    }}

    Pamiętaj o polach:
    - function.parameters.required,
    - function.parameters.additionalProperties
    - function.strict

    Pamiętaj, że w OpenAI Tools Schema parametry są opisane za pomocą 'type' i 'description'.
    
    Description dla pól możesz musieć wyciągnąć z opisu parametrów w głównym description opisu toola w Model Context Protocol.
    ---
    {mcp_tools_json}
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Jesteś ekspertem od przetwarzania dokumentów JSON"},
        {"role": "user", "content": conversion_prompt},
    ],
    response_format={
        "type": "json_object"
    },
    temperature=0.0,
    max_tokens=10000
)


openai_tools_json = response.choices[0].message.content

In [ ]:
print(json.dumps(json.loads(openai_tools_json), indent=4))

In [ ]:
async def handle_function_call(mcp_url, function_name, arguments):
    """
    Obsługuje wywołanie funkcji na serwerze MCP.

    Parametry:
    mcp_url (str): URL serwera MCP.
    function_name (str): Nazwa funkcji, którą należy wywołać.
    arguments (dict): Argumenty przekazywane do funkcji.

    Zwraca:
    str: Wynik wywołania funkcji w postaci tekstu.
    """
    print(f"<function_call> Function: {function_name}, Arguments: {arguments}")

    # Nawiązanie połączenia SSE
    async with sse_client(url=mcp_url) as (read, write):
        # Utworzenie sesji klienta
        async with ClientSession(read, write) as session:
            # Inicjalizacja połączenia
            await session.initialize()
            
            # Wywołanie funkcji na serwerze MCP
            result = await session.call_tool(function_name, arguments=arguments)

    return result.content[0].text

In [ ]:
await handle_function_call(mcp_url=mcp_url, function_name="add", arguments={"a": "2", "b": "3"})

In [ ]:
async def execute_tool_calls(response, messages):
    """
    Wykonuje funkcje zlecone przez model i aktualizuje historię wiadomości.

    Parametry:
    response (openai.types.chat.chat_completion.ChatCompletion): Odpowiedź od modelu zawierająca zlecenia funkcji.
    messages (list): Historia wiadomości.

    Zwraca:
    list: Zaktualizowana historia wiadomości.
    """

    # dodajemy odpowiedź, w tym ew. prośby o wywołanie funkcji, do historii wiadomości
    messages.append(response.choices[0].message.model_dump())

    for tool_call in response.choices[0].message.tool_calls:
        # pobieramy nazwę funkcji i argumenty z odpowiedzi od modelu
        name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)

        # wywołanie funkcji na podstawie pobranej nazwy funkcji i argumentów
        result = await handle_function_call(mcp_url, name, args)

        # dodajemy wynik funkcji do historii wiadomości
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result
        })

    return messages

In [ ]:
# Wczytanie listy narzędzi z serwera MCP
tools = json.loads(openai_tools_json)['tools']

async def process_user_command(user_command):
    """
    Przetwarza polecenie użytkownika z użyciem GPT-4o oraz funkcji odczytanych z serwera MCP.

    Parametry:
    user_command (str): Polecenie użytkownika.

    Zwraca:
    str: Odpowiedź na polecenie użytkownika.
    """

    messages = [
        {"role": "system", "content": "Jesteś pomocnym asystentem."},
        {"role": "user", "content": user_command}
    ]
    
    while True:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0.0,
            max_tokens=16000
        )
        
        message = response.choices[0].message
        
        # Jeśli model żąda wykonania funkcji, wykonujemy je i aktualizujemy historię wiadomości
        # (nastąpi kolejne wykonanie pętli zawierające request do modelu, w którym przekażemy wyniki wykonania funkcji)
        if getattr(message, "tool_calls", None):
            messages = await execute_tool_calls(response, messages)
        else:
            # Gdy nie ma już żądań funkcji, zwracamy ostateczną odpowiedź modelu. Ale najpierw uzupełniamy historię wiadomości
            messages.append(response.choices[0].message.model_dump())

            return message.content, messages

In [ ]:
result, messages = await process_user_command("Ile wynosi: 3+ln(3)*3?")

In [ ]:
print(result)

In [ ]:
messages

## Pydantic AI

Komunikacja z serwerem MCP (pobieranie listy funkcji, tłumaczenie ich opisu oraz wykonywanie funkcji) jest obsługiwana przez bibliotekę.

In [ ]:
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.mcp import MCPServerHTTP
import os
import nest_asyncio
nest_asyncio.apply()

In [ ]:
mcp_url = "http://mcp:8801/sse"

In [ ]:
mcp_server = MCPServerHTTP(url=mcp_url)

api_key = os.getenv('OPENAI_API_KEY')
provider = OpenAIProvider(api_key=api_key)
model = OpenAIModel('gpt-4o', provider=provider)

agent = Agent(
    model = model,
    system_prompt='Bądź zwięzły, odpowiadaj jednym zdaniem.',
    mcp_servers=[mcp_server]
)

# Albo Gemini jako alternatywa
# agent = Agent(
#     model = 'gemini-2.5-pro-exp-03-25',
#     system_prompt='Użyj dostępnych narzędzi MCP aby obsłużyć zapytanie.',
#     mcp_servers=[mcp_server]
# )

async with agent.run_mcp_servers():
    result = agent.run_sync('Ile wynosi: 3+ln(3)*3?')

print(result.output)

## Open AI agents

Podobnie jak w przypadku Pydantic AI, Open AI Agents automatyzuje obsługę komunikacji z serwerem MCP

In [ ]:
from agents import Agent, Runner
from agents.mcp import MCPServerSse

In [ ]:
mcp_server = MCPServerSse(params={"url": "http://mcp:8801/sse",})

await mcp_server.connect()

In [ ]:
# Agent pobiera klucz ze zmiennej środowiskowej
# ale można go ustawić bezpośrednio w kodzie:
# import os
# os.environ["OPENAI_API_KEY"] = 'api_key'

agent = Agent(
    name="Agent MCP",
    model="gpt-4.5-preview",
    instructions="Użyj dostępnych narzędzi MCP aby obsłużyć zapytanie.",
    mcp_servers=[mcp_server]
)

result = await Runner.run(starting_agent=agent, input="Ile wynosi: 3+ln(3)*3?")

print(result.final_output)

I wersja z bezpośrednim przekazaniem klucza API 

In [ ]:
from agents import OpenAIChatCompletionsModel
from openai import AsyncOpenAI
import os

api_key = os.getenv('OPENAI_API_KEY')

client = AsyncOpenAI(api_key=api_key)
model = OpenAIChatCompletionsModel(model="gpt-4o", openai_client=client)

agent = Agent(
    name="Agent MCP",
    model=model,
    instructions="Użyj dostępnych narzędzi MCP aby obsłużyć zapytanie.",
    mcp_servers=[mcp_server]
)

result = await Runner.run(starting_agent=agent, input="Ile wynosi: 3+ln(3)*3?")

print(result.final_output)

## Zasoby w MCP

Oprócz narzędzi (funkcji), Model Context Protocol umożliwia również dostęp do zasobów. W przeciwieństwie do narzędzi, które wykonują jakieś obliczenia lub operacje, zasoby reprezentują dane statyczne lub dynamicznie generowane treści.

Na naszym serwerze MCP zdefiniowane są dwa zasoby:
1. `resource://math_symbols` - zwraca listę wszystkich dostępnych kluczy symboli matematycznych
2. `resource://math_symbol/{symbol_name}` - zwraca konkretny symbol matematyczny dla podanej nazwy

Przyjrzyjmy się, jak możemy odczytać te zasoby z serwera MCP.

In [ ]:
# Adres serwera MCP
mcp_url = "http://mcp:8801/sse"

# Aby korzysztać z lokalnego serwera MCP, uruchom go w terminalu wspisując:
# > export MCP_PORT=1234
# > python mcp_scripts/math_functions.py 
# i odkomentuj poniższą linię
# mcp_url = "http://localhost:1234/sse"

In [ ]:
# funkcja pobierająca listę zasobów z serwera MCP
async def mcp_list_resources(mcp_url):
    """
    Pobiera listę dostępnych zasobów z serwera MCP.

    Parametry:
    mcp_url (str): URL serwera MCP.

    Zwraca:
    list: Lista dostępnych zasobów w formacie zwracanym przez serwer MCP.
    """
    # Nawiązanie połączenia SSE
    async with sse_client(url=mcp_url) as (read, write):
        # Utworzenie sesji klienta
        async with ClientSession(read, write) as session:
            # Inicjalizacja połączenia
            await session.initialize()

            # Pobranie listy dostępnych zasobów
            resources_response = await session.list_resources()

            return resources_response

In [ ]:
# Pobranie listy dostępnych zasobów
resources = await mcp_list_resources(mcp_url)
resources

Jak widzimy, metoda `list_resources()` zwraca tylko statyczne zasoby (w naszym przypadku `resource://math_symbols`), ale nie wyświetla zasobów dynamicznych (z parametrami), takich jak `resource://math_symbol/{symbol_name}`.

Jest to standardowe zachowanie protokołu MCP - zasoby dynamiczne nie są zwracane w listach zasobów, ponieważ wymagają podania parametrów do określenia konkretnego zasobu.

Aby dowiedzieć się o dostępnych zasobach dynamicznych, należy zapoznać się z dokumentacją serwera MCP lub jego kodem źródłowym.

In [ ]:
async def mcp_get_resource(mcp_url, resource_uri):
    """
    Pobiera zawartość zasobu z serwera MCP.

    Parametry:
    mcp_url (str): URL serwera MCP.
    resource_uri (str): URI zasobu do pobrania.

    Zwraca:
    any: Zawartość zasobu.
    """
    # Nawiązanie połączenia SSE
    async with sse_client(url=mcp_url) as (read, write):
        # Utworzenie sesji klienta
        async with ClientSession(read, write) as session:
            # Inicjalizacja połączenia
            await session.initialize()

            # Pobranie zawartości zasobu
            resource_response = await session.read_resource(resource_uri)

            return resource_response.contents[0].text

# Odczytanie zasobu statycznego z listą symboli matematycznych
math_symbols_text = await mcp_get_resource(mcp_url, "resource://math_symbols")
math_symbols = json.loads(math_symbols_text)

print(math_symbols)

In [ ]:
# Odczytanie zasobu dynamicznego dla konkretnego symbolu matematycznego
symbol_name = "pi"  # możesz zmienić na dowolny inny symbol z listy
symbol = await mcp_get_resource(mcp_url, f"resource://math_symbols/{symbol_name}")

print(f"Symbol '{symbol_name}': {symbol}")

In [ ]:
# Przykład odczytania kilku symboli matematycznych
symbols_to_get = ["pi", "integral", "square_root", "infinity"]

for symbol_name in symbols_to_get:
    symbol = await mcp_get_resource(mcp_url, f"resource://math_symbols/{symbol_name}")
    print(f"Symbol '{symbol_name}': {symbol}")

### Podsumowanie pracy z zasobami MCP

Zasoby MCP są użyteczne do udostępniania statycznych lub dynamicznie generowanych danych dla modeli językowych:

1. **Zasoby statyczne** (np. `resource://math_symbols`) mogą zawierać listy, słowniki lub inne dane, które są zawsze takie same.

2. **Zasoby dynamiczne** (np. `resource://math_symbol/{symbol_name}`) mogą generować dane na podstawie parametrów zawartych w URI.

3. **Metoda `list_resources()`** zwraca tylko statyczne zasoby, nie pokazuje zasobów dynamicznych z parametrami.

4. Aby odczytać zasób, używamy metody `get_resource(resource_uri)` z odpowiednim URI.

Zasoby mogą być wykorzystywane przez modele językowe do pobierania danych referencyjnych, konfiguracji, lub innych informacji bez konieczności używania function callingu.